In [1]:
import xml.etree.ElementTree as ET
from xml.sax.saxutils import escape
import pprint
import json
import xmltodict
import pandas as pd
import os

In [2]:
def extractData(inp):
    data=json.loads(jsonString)
    data=data['CHART']['RACE']
    horsieData=[]
    for race in data:
        run={}
        run['distance']=float(race['DISTANCE'])
        if(race['COURSE_ID']=='D'):
            run['track']=1
        else:
            run['track']=0
        run['fr1']=float(race['FRACTION_1'])
        run['fr2']=float(race['FRACTION_2'])
        run['fr3']=float(race['FRACTION_3'])
        run['fr4']=float(race['FRACTION_4'])
        run['fr5']=float(race['FRACTION_5'])
        run['frw']=float(race['WIN_TIME'])
        horse={}
        for entry in race['ENTRY']:
            horse['pp_start']=int(entry['START_POSITION'])
            horse['pp_finish']=int(entry['OFFICIAL_FIN'])
            if(int(entry['OFFICIAL_FIN'])<4):
                horse['winloss']=1
            else:
                horse['winloss']=0
            if(int(entry['OFFICIAL_FIN'])==1):
                horse['first']=1
                horse['second']=0
                horse['third']=0
                horse['loser']=0
            elif(int(entry['OFFICIAL_FIN'])==2):
                horse['first']=0
                horse['second']=1
                horse['third']=0
                horse['loser']=0
            elif(int(entry['OFFICIAL_FIN'])==3):
                horse['first']=0
                horse['second']=0
                horse['third']=1
                horse['loser']=0
            else:
                horse['first']=0
                horse['second']=0
                horse['third']=0
                horse['loser']=1
            horse['age']=int(entry['AGE'])
            horse['speed_rating']=int(entry['SPEED_RATING'])
            horse['horse']=str(entry['AXCISKEY'])
            horse['jockey name']=str(entry['JOCKEY']['FIRST_NAME'])+' '+str(entry['JOCKEY']['LAST_NAME'])
            horse['trainer']=str(entry['TRAINER']['KEY'])
            horse['odds']=float(entry['DOLLAR_ODDS'])
            call={}
            for poc in entry['POINT_OF_CALL']:
                if(poc['@WHICH']=='1'):
                    call['poc1']=float(poc['LENGTHS'])
    #                 row['poc1p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='2'):
                    call['poc2']=float(poc['LENGTHS'])
    #                 row['poc2p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='3'):
                    call['poc3']=float(poc['LENGTHS'])
    #                 row['poc3p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='4'):
                    call['poc4']=float(poc['LENGTHS'])
    #                 row['poc4p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='5'):
                    call['poc5']=float(poc['LENGTHS'])
    #                 row['poc5p']=int(poc['POSITION'])
                elif(poc['@WHICH']=='FINAL'):
                    call['pocf']=float(poc['LENGTHS'])
    #                 row['pocfp']=int(poc['POSITION'])
            outp={**run, **horse, **call}
            horsieData.append(outp)
    return horsieData

In [3]:
horsieData=[]

for x in os.listdir('resources/races'):
    file='resources/races/'+str(x)
    with open(file, 'r') as f:
        xmlString = f.read()
    jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
    horsieData+=extractData(jsonString)

In [4]:
df=pd.DataFrame(horsieData)
df.head()

,age,distance,first,fr1,fr2,fr3,fr4,fr5,frw,horse,...,poc5,pocf,pp_finish,pp_start,second,speed_rating,third,track,trainer,winloss
0,4,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049053050051052058055056,...,0.0,4.5,4,6,0,107,0,1,957712,0
1,6,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049051050051061055061063,...,2.0,2.8,3,2,0,110,1,1,250678,1
2,5,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049052050052052057061059,...,2.1,6.3,6,1,0,104,0,1,38601,0
3,7,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049050050052060060061060,...,2.7,5.5,5,7,0,105,0,1,224590,0
4,4,800.0,1,22.8,46.58,71.25,84.5,0.0,97.5,049053050051054056057063,...,2.2,0.0,1,5,0,114,0,1,6363,1


In [5]:
# file='resources/DMR20180718TCH.xml'
# with open(file, 'r') as f:
#     xmlString = f.read()
# jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
# print(jsonString)

In [20]:
horseLength=8
furlong=660
dist1=1320
dist2=2640
dist3=3960
df['speed1']=((dist1-(horseLength*df['poc1'].astype(float)))/df['fr1'].astype(float)).round(2)
df['speed2']=((dist2-(horseLength*df['poc2'].astype(float)))/df['fr2'].astype(float)).round(2)
# df['speed3']=((dist3-(horseLength*df['poc3'].astype(float)))/df['fr3'].astype(float)).round(2)
df['speedf']=(((df['distance'].astype(float)/100*furlong)-(horseLength*df['pocf'].astype(float)))/df['frw'].astype(float)).round(2)
df.head()

,age,distance,first,fr1,fr2,fr3,fr4,fr5,frw,horse,...,pp_start,second,speed_rating,third,track,trainer,winloss,speed1,speed2,speedf
0,4,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049053050051052058055056,...,6,0,107,0,1,957712,0,57.89,56.68,53.78
1,6,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049051050051061055061063,...,2,0,110,1,1,250678,1,56.81,55.99,53.92
2,5,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049052050052052057061059,...,1,0,104,0,1,38601,0,57.72,56.59,53.64
3,7,800.0,0,22.8,46.58,71.25,84.5,0.0,97.5,049050050052060060061060,...,7,0,105,0,1,224590,0,57.02,56.25,53.70
4,4,800.0,1,22.8,46.58,71.25,84.5,0.0,97.5,049053050051054056057063,...,5,0,114,0,1,6363,1,56.28,55.82,54.15


In [21]:
# data=pd.DataFrame(df[['distance','pp_start','speed1','speed2','speed3','speedf','pp_finish']])
data=pd.DataFrame(df[['distance','track','age','jockey name','trainer','speed_rating','odds','pp_start','speed1','speed2','speedf','pp_finish','winloss']])
data.head()

,distance,track,age,jockey name,trainer,speed_rating,odds,pp_start,speed1,speed2,speedf,pp_finish,winloss
0,800.0,1,4,William Antongeorgi,957712,107,4.4,6,57.89,56.68,53.78,4,0
1,800.0,1,6,Kyle Frey,250678,110,13.9,2,56.81,55.99,53.92,3,1
2,800.0,1,5,Drayden Van Dyke,38601,104,11.1,1,57.72,56.59,53.64,6,0
3,800.0,1,7,Joseph Talamo,224590,105,3.9,7,57.02,56.25,53.70,5,0
4,800.0,1,4,Rafael Bejarano,6363,114,3.1,5,56.28,55.82,54.15,1,1


In [22]:
# data2=pd.DataFrame(df[['distance','track','age','jockey','trainer','speed_rating','odds','pp_start','speed1','speed2','speedf','pp_finish','first','second','third','loser']])
# data2.head()

In [23]:
# data.to_csv('output/horseData_20180802v1.0.csv', index=False)
# data2.to_csv('output/horseData_20180801v2.csv', index=False)

In [24]:
# data[data['winloss']==1].sort_values(by='odds', ascending=False)

In [25]:
jockey=pd.read_csv('resources/jockies/jockeystats.csv')
jockey=jockey.rename(columns={'jockey_name':'jockey name'})
jockey.sort_values(by='jockey name')

,jockey name,j_starts,j_first,j_second,j_third,j_total_win_amt,j_win_perc,j_top_3,j_top_3_perc
33,Abel Cedillo,1,0,0,0,6000,0.00,0,0.00
24,Agapito Delgadillo,9,1,1,2,34975,0.11,4,0.44
38,Alex Jimenez,2,0,0,0,745,0.00,0,0.00
23,Alonso Quinonez,28,3,1,1,66200,0.11,5,0.18
6,Asa Espinoza,63,9,6,5,314435,0.14,20,0.32
7,Assael Espinoza,63,9,6,5,314435,0.14,20,0.32
39,Barrington Harvey,2,0,0,0,690,0.00,0,0.00
27,Brayan Pena,12,1,1,2,25830,0.08,4,0.33
30,Brice Blanc,7,0,0,1,11685,0.00,1,0.14
14,Corey Nakatani,19,1,4,4,168321,0.05,9,0.47


In [26]:
hj=pd.merge(data,jockey, how='left', left_on='jockey name', right_on='jockey name')
hj.sort_values(by='jockey name').head()

,distance,track,age,jockey name,trainer,speed_rating,odds,pp_start,speed1,speed2,...,pp_finish,winloss,j_starts,j_first,j_second,j_third,j_total_win_amt,j_win_perc,j_top_3,j_top_3_perc
789,700.0,1,3,Abel Cedillo,804,75,33.2,2,59.25,57.78,...,5,0,1,0,0,0,6000,0.00,0,0.00
397,800.0,0,7,Agapito Delgadillo,624,99,35.5,9,56.67,57.03,...,11,0,9,1,1,2,34975,0.11,4,0.44
394,500.0,1,2,Agapito Delgadillo,94,67,4.9,4,57.62,55.25,...,3,1,9,1,1,2,34975,0.11,4,0.44
602,850.0,0,4,Agapito Delgadillo,624,91,20.9,1,57.90,55.67,...,4,0,9,1,1,2,34975,0.11,4,0.44
24,600.0,1,4,Agapito Delgadillo,1781,96,14.3,1,58.45,57.25,...,3,1,9,1,1,2,34975,0.11,4,0.44


In [27]:
hj.to_csv('output/horseData_20180803v1.0.csv', index=False)